# Finetune ruGPT3Small on gratters

---



## Install enviroment

In [ ]:
!pip3 install urllib3==1.25.4

In [ ]:
!pip3 install 

In [ ]:
!pip3 install transformers==2.8.0

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

--2021-02-17 20:13:37--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34150 (33K) [text/plain]
Saving to: ‘pretrain_transformers.py.2’

pretrain_transforme 100%[===================>]  33.35K  --.-KB/s    in 0.001s  

2021-02-17 20:13:37 (23.0 MB/s) - ‘pretrain_transformers.py.2’ saved [34150/34150]



In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

--2021-02-17 20:13:37--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10474 (10K) [text/plain]
Saving to: ‘generate_transformers.py.2’

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

2021-02-17 20:13:38 (119 MB/s) - ‘generate_transformers.py.2’ saved [10474/10474]



In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [ ]:
!sh setup.sh

fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-t1fvdcl5
Created temporary directory: /tmp/pip-req-tracker-im8ej2lb
Created requirements tracker '/tmp/pip-req-tracker-im8ej2lb'
Created temporary directory: /tmp/pip-install-k6bv_lb0
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-hun_hsae
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-im8ej2lb'
    Running setup.py (path:/tmp/pip-req-build-hun_hsae/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.7.0+cu101


    running egg_info
    creating /tmp/pip-req-build-hun_hsae/pip-egg-info/apex.egg-info

## Add data to colab
Add essays file from google dirve:
* Add [file](https://drive.google.com/file/d/10ZsjTeaoihYA80n1G40O5YZmaGw0yOXk/view?usp=sharing) to your own drive
* And mount drive to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
proza_path = "drive/My Drive/Colab Notebooks/HB generator/proza.csv"
stih_path = "drive/My Drive/Colab Notebooks/HB generator/stih.csv"
!ls "$proza_path"

'drive/My Drive/Colab Notebooks/HB generator/proza.csv'


In [ ]:
import pandas as pd 
df_proza = pd.read_csv(proza_path, names = ['N', 'text'], header = 0)  
df_stih =pd.read_csv(stih_path, names = ['N', 'text'], header = 0)  

In [ ]:
from sklearn.model_selection import train_test_split

Proza_train, Proza_test = train_test_split(df_proza, test_size = 0.2, random_state=124)
Stih_train, Stih_test = train_test_split(df_stih, test_size = 0.2, random_state=124)

In [ ]:
import io, ast
with io.open("train.txt", 'w', encoding="utf-8") as f:
    for index, row in Proza_train.iterrows():
        f.write('<s>Тип: Проза\nТекст: ' + ast.literal_eval(row.text).decode("utf-8") + '</s>\n')
    for index, row in Stih_train.iterrows():
        f.write('<s>Тип: Стих\nТекст: ' + ast.literal_eval(row.text).decode("utf-8").replace('\n','\g') + '</s>\n') 
        
with io.open("valid.txt", 'w', encoding="utf-8") as f:
    for index, row in Proza_test.iterrows():
        f.write('<s>Тип: Проза\nТекст: ' + ast.literal_eval(row.text).decode("utf-8") + '</s>\n')
    for index, row in Stih_test.iterrows():
        f.write('<s>Тип: Стих\nТекст: ' + ast.literal_eval(row.text).decode("utf-8").replace('\n','\g') + '</s>\n')

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [ ]:
!python pretrain_transformers.py \
    --output_dir=hb_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 1028 \
    --overwrite_output_dir

Выходные данные были обрезаны до нескольких последних строк (5000).
Iteration:  92% 1070/1162 [06:25<00:32,  2.85it/s]
Iteration:  92% 1071/1162 [06:26<00:31,  2.85it/s]
Iteration:  92% 1072/1162 [06:26<00:31,  2.85it/s]
Iteration:  92% 1073/1162 [06:26<00:31,  2.83it/s]
Iteration:  92% 1074/1162 [06:27<00:31,  2.81it/s]
Iteration:  93% 1075/1162 [06:27<00:30,  2.85it/s]
Iteration:  93% 1076/1162 [06:27<00:30,  2.85it/s]
Iteration:  93% 1077/1162 [06:28<00:29,  2.85it/s]
Iteration:  93% 1078/1162 [06:28<00:29,  2.84it/s]
Iteration:  93% 1079/1162 [06:28<00:29,  2.84it/s]
Iteration:  93% 1080/1162 [06:29<00:28,  2.85it/s]
Iteration:  93% 1081/1162 [06:29<00:28,  2.84it/s]
Iteration:  93% 1082/1162 [06:30<00:28,  2.85it/s]
Iteration:  93% 1083/1162 [06:30<00:27,  2.85it/s]
Iteration:  93% 1084/1162 [06:30<00:27,  2.82it/s]
Iteration:  93% 1085/1162 [06:31<00:27,  2.82it/s]
Iteration:  93% 1086/1162 [06:31<00:26,  2.82it/s]
Iteration:  94% 1087/1162 [06:31<00:26,  2.84it/s]
Iteration:  94

## Check our model

In [ ]:
## Select topic
for idx in range(len(valid)):
  if "образует его общество" in valid[idx]:
    break

In [ ]:
valid[idx]

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=hb_model \
    --k=5 \
    --p=0.95 \
    --length=200

^C
